#### Loading the images

In [197]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
from skimage.feature import hog
%matplotlib inline


In [198]:
#----------------------------------------------
# Get Spatial binning return 32*32 features
#----------------------------------------------

def spatial_bin_feat(img,color_space = 'RGB',size = (32,32)):
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_img = cv2.cvtColor(img,cv2.COLOR_RGB2HSV)
        elif color_space == 'HLS':
            feature_img = cv2.cvtColor(img,cv2.COLOR_RGB2HLS)
            feature_img = feature_img[:,:,1]
#             plt.title(feature_img[:,:,1].shape)
#             plt.imshow(feature_img[:,:,1],cmap = 'gray')
    else:
        feature_img = np.copy(img)
#     print(feature_img.shape)
    features = cv2.resize(feature_img,size).ravel()
   # print(features.shape)
    return features

#----------------------------------------------
# Get Color histogram features
#----------------------------------------------
def color_hist(img,nbins =32,bins_range = (0,256)):
    # histogram if RGB channels
    rhist = np.histogram(img[:,:,0],bins = nbins,range = bins_range)
    ghist = np.histogram(img[:,:,1],bins = nbins,range = bins_range)
    bhist = np.histogram(img[:,:,2],bins = nbins,range = bins_range)
    
    h_features = np.concatenate((rhist[0],ghist[0],bhist[0]))
    return h_features

#----------------------------------------------
# Get HOG features
#----------------------------------------------
def hog_features(img,orient,pix_per_cell,cell_per_block,vis = False ,feature_vec = True):
    ret_list = hog(img,orientations = orient ,pixels_per_cell = (pix_per_cell,pix_per_cell),cells_per_block = (cell_per_block,cell_per_block),block_norm= 'L2-Hys', transform_sqrt=False, visualise= vis, feature_vector= feature_vec)
    # name returns explicitly
    hog_features = ret_list[0]
    if vis:
        hog_image = ret_list[1]
        return hog_features.ravel(), hog_image
    else:
        return hog_features.ravel()
    
#----------------------------------------------
# Get Combined features
#----------------------------------------------
def get_all_features(t_img):
    sb_features = spatial_bin_feat(t_img,'HLS')
    # print(sb_features.size)# 1024
    hist_features = color_hist(t_img)
    # print(hist_features.size) # 96
    gray = cv2.cvtColor(t_img,cv2.COLOR_RGB2GRAY)
    h_features, hog_image = hog_features(gray, orient= 9, 
                            pix_per_cell= 8, cell_per_block= 2, 
                            vis=True, feature_vec=False)

    # print(hog_features.size) #1764
    # print(hog_features.shape) #(1764,)
    comb_feature = np.concatenate((sb_features,hist_features,h_features))
#     print(comb_feature)
    return comb_feature

In [199]:
non_vehicle_paths = ["non-vehicles/Extras/*.png", "non-vehicles/GTI/*.png"]

vehicle_paths = ["vehicles/GTI_Far/*.png", "vehicles/GTI_Left/*.png", "vehicles/GTI_Right/*.png",
                    "vehicles/GTI_MiddleClose/*.png", "vehicles/KITTI_extracted/*.png"]

non_vehicle_filenames = []
vehicle_filenames = []

for path in non_vehicle_paths:
    non_vehicle_filenames += glob.glob(path)
    
for path in vehicle_paths:
    vehicle_filenames += glob.glob(path)
    
# print(non_vehicle_filenames[0])
# print(vehicle_filenames[0])

In [189]:
# print("#Vehicles :",len(vehicle_filenames),"#Non-Vehicles:",len(non_vehicle_filenames))
# ## #Vehicles : 8792 #Non-Vehicles: 8968

# nv_img = plt.imread(non_vehicle_filenames[10])
# v_img = plt.imread(vehicle_filenames[10])
# plt.subplot(1,2,1)
# plt.title("Non vehicle")
# plt.imshow(nv_img)
# plt.subplot(1,2,2)
# plt.title("Vehicle")
# plt.imshow(v_img)

In [201]:
# print(np.max(nv_img),np.min(nv_img)) #0.8117647 0.015686275
# print(v_img.shape) #(64, 64, 3)
all_img_features = []
for idx in range(len(vehicle_filenames)):
    # to get all the feature
    v_img = plt.imread(vehicle_filenames[idx])
    img_feature = get_all_features(v_img)
    img_feature = img_feature.tolist()
    all_img_features.append(img_feature)
# print(all_img_features.size)
print(len(all_img_features))

for nv_idx in range(len(non_vehicle_filenames)):
    # to get all the feature
    nv_img = plt.imread(non_vehicle_filenames[nv_idx])
    img_feature = get_all_features(nv_img)
    img_feature = img_feature.tolist()
    all_img_features.append(img_feature)
    
# one hot encoded vectors is_car = 1 if its a car
is_car = np.ones(len(vehicle_filenames))
is_car = is_car.tolist()
# print(len(is_car))
non_car = np.zeros(len(non_vehicle_filenames))
non_car = non_car.tolist()
is_car += non_car
print(len(is_car))

8792
17760


In [205]:
# writing feature vectors in pickle file
import pickle
output_f = open('features.pkl','wb')
output_h = open('onehot.pkl','wb')
pickle.dump(all_img_features,output_f)
pickle.dump(is_car,output_h)

In [213]:
f_file = open('features.pkl','rb')
h_file = open('onehot.pkl','rb')
X_pkl = pickle.load(f_file)
Y_pkl = pickle.load(h_file)
# print(len(X_train)) #17760
# print(len(Y_train)) #17760
X = np.vstack(X_pkl).astype(np.float64)
y = np.hstack(Y_pkl)

In [215]:
#-----------------------------
# Training the SVN model
#-----------------------------
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVC

rand_state = np.random.randint(0,100)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = rand_state)

# fit a per column scalar
X_scalar = StandardScaler().fit(X_train)
#Apply the scalar  to X
scaled_X_train = X_scalar.transform(X_train)
scaled_X_test = X_scalar.transform(X_test)
#print(scaled_X_test)

svc = LinearSVC()
svc.fit(scaled_X_train,y_train)



LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [219]:
print("Train Accuracy :",svc.score(scaled_X_train,y_train))
# Train Accuracy : 0.9985923423423423
print("Test Accuracy :",svc.score(scaled_X_test,y_test))

Train Accuracy : 0.9985923423423423
Test Accuracy : 0.9293355855855856


In [177]:
# a = np.array([1,23,44])
# a = a.tolist()
# print(type(a))
# x = []
# t = [1,2]
# p = [1,3]
# x.append(t)
# x.append(p)
# print(x)